In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch as t
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_distances
from pathlib import Path
from scipy.spatial.distance import cdist

In [ ]:
# load data:
data_dir = Path('/Users/thomasbush/Documents/Vault/DSS_Tilburg/data')
features = np.load(data_dir / 'features.npy')

# load keyframes kmean:
keyframes_kmean = data_dir / 'keyframes/_2025-04-22 00:25:47.432414_keyframes.pth'
keyframes_idx_fps = data_dir / 'keyframes_output/keyframes_fps_500.npy'

# load idx fps
idx_fps = np.load(keyframes_idx_fps)

#load keyframes kmean:
data = t.load(keyframes_kmean, map_location='cpu', weights_only=False)
keyframes_kmean = data['keyframes']
kmean_idx = data['keyframe_idx']

In [ ]:
# load video to display frames
from utils.utils import loadVideoArray
vid_path = '/Users/thomasbush/Documents/Vault/Iurilli_lab/3d_tracking/data/multicam_video_2024-07-22T10_19_22_cropped_20250325101012/multicam_video_2024-07-22T10_19_22_mirror-left.avi.mp4'
vid = loadVideoArray(vid_path)

## Error analysis:

Display frames that are further from each other and that are close to each other for both clustering methods

In [ ]:
pca = PCA(n_components=2)
embedding = pca.fit_transform(features)

### FPS analysis

In [ ]:
plt.figure(figsize=(12, 9))

# Plot all frames in light gray
plt.scatter(embedding[:, 0], embedding[:, 1], c='lightgray', s=10, alpha=0.5, label="All frames")

# Highlight FPS-selected keyframes
plt.scatter(embedding[idx_fps, 0], embedding[idx_fps, 1],
            c='black', s=60, marker='*', label="Keyframes (FPS)", edgecolors='white')

# Annotate a few FPS keyframe indices
for idx in idx_fps[:20]:  # only first 20 to avoid clutter
    plt.annotate(str(idx), (embedding[idx, 0], embedding[idx, 1]), fontsize=8, color='black')

# Labels and formatting
plt.xlabel("PCA Dimension 1")
plt.ylabel("PCA Dimension 2")
plt.title("Keyframe Selection via Farthest Point Sampling (FPS)")
plt.legend()
plt.tight_layout()

# Optional: save
save_dir = data_dir / 'keyframes'
plt.savefig(save_dir / "keyframes_fps_clustering.svg", format="svg", bbox_inches="tight", dpi=300)

plt.show()

In [ ]:
# select three similar frames from the pca: 773, 123 and two far away: 0, 163

f, axs = plt.subplots(1,2)
axs[0].imshow(vid[0], cmap='gray')
axs[1].imshow(vid[163], cmap='gray')

plt.show()

### Kmeans error analysis

In [ ]:
import matplotlib.cm as cm
from sklearn.metrics import pairwise_distances_argmin_min
# Apply KMeans clustering
features_pca = embedding
kmeans = KMeans(n_clusters=500, random_state=42)
kmeans.fit(features_pca)
closest_indices, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, features_pca)

# Assign clusters to all points
labels = kmeans.predict(features_pca)
colors = cm.tab20(labels.astype(float) % 20 / 20)

# Plot
plt.figure(figsize=(12, 9))
plt.scatter(features_pca[:, 0], features_pca[:, 1], c=colors, s=10, alpha=0.5, label="All frames")
plt.scatter(features_pca[closest_indices, 0], features_pca[closest_indices, 1],
            c='black', s=60, marker='*', label="Keyframes", edgecolors='white')

# Annotate a few keyframe indices for draw.io use
for idx in closest_indices[:20]:  # Only first 20 to avoid clutter
    plt.annotate(str(idx), (features_pca[idx, 0], features_pca[idx, 1]), fontsize=8, color='black')

plt.xlabel("PCA Dimension 1")
plt.ylabel("PCA Dimension 2")
plt.title("Keyframe Selection via KMeans Clustering")
plt.legend()
plt.tight_layout()
# plt.savefig(save_path / 'pca_kmean_clustering.svg', format="svg", bbox_inches="tight", dpi=300)
plt.show()


In [ ]:
from sklearn.metrics import mean_squared_error

def plot_two_frames_side_by_side(vid, idx1, idx2, titles=None, cmap='gray', save_path=None):
    """
    Show two video frames side by side and display MSE between them.

    Args:
        vid: numpy array of video frames, shape (N, H, W) or (N, H, W, 3)
        idx1, idx2: indices of frames to compare
        titles: list or tuple of two strings for titles
        cmap: color map to use ('gray' or None)
        save_path: optional path to save the figure
    """
    frame1 = vid[idx1]
    frame2 = vid[idx2]

    # Flatten and compute MSE
    mse = mean_squared_error(frame1.flatten(), frame2.flatten())

    fig, axs = plt.subplots(1, 2, figsize=(8, 4))
    
    axs[0].imshow(frame1, cmap=cmap)
    axs[0].axis('off')
    axs[0].set_title(titles[0] if titles else f"Frame {idx1}")
    
    axs[1].imshow(frame2, cmap=cmap)
    axs[1].axis('off')
    axs[1].set_title(titles[1] if titles else f"Frame {idx2}")

    # Add MSE as a subtitle
    plt.suptitle(f"MSE between frames: {mse:.4f}", fontsize=12)
    plt.tight_layout(rect=[0, 0, 1, 0.95])  # leave room for suptitle

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
# Assuming vid is your frame array and frames are grayscale
plot_two_frames_side_by_side(vid, idx1=773, idx2=123, save_path=data_dir / 'keyframes/closefps.png')

In [ ]:
plot_two_frames_side_by_side(vid, idx1=0, idx2=163, save_path=data_dir / 'keyframes/furtherfps.png')

In [ ]:
plot_two_frames_side_by_side(vid, idx1=33826, idx2=16779, save_path=data_dir / 'keyframes/closekmeans.png')

In [ ]:
plot_two_frames_side_by_side(vid, idx1=11906, idx2=30501,   save_path=data_dir / 'keyframes/furtherkmeans.png')